In [1]:
dir_data = '../data/skin-cancer'
dir_ckpt = './outputs/ckpt'
dir_figure = './outputs/figures'
dir_target = './outputs/gradcam_guided_images'

seed = 0
method = 'betti-curve'
n_bins = 100

input_shape = (28, 28, 1) 
n_classes = 7

In [2]:
import sys

sys.path.append('../')

from cnntdanet.utils import seed_all
from cnntdanet.tda import get_tda_pipeline
from cnntdanet.gradcam import GradCAMOnCNN, GradCAMOnCNNTDANet

import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


In [3]:
seed_all(seed)

## Loading dataset

In [ ]:
# Preparing dataset
df = pd.read_csv(os.path.join(dir_data, 'hmnist_28_28_L.csv'))

X = df.drop('label', axis=1).values.reshape((-1, *input_shape))
y = df['label'].values

X = X.astype(np.float32)
X /= 255.0

y = y.astype(np.int64)

## TDA features
pipeline = get_tda_pipeline(method=method, n_bins=n_bins)

X_tda = pipeline.fit_transform(X)
X_tda = X_tda.astype(np.float32)

## Loading model

In [ ]:
cnn = tf.keras.models.load_model(os.path.join(dir_ckpt, 'hmnist_cnn_seed0'))
cnn_tda = tf.keras.models.load_model(os.path.join(dir_ckpt, 'hmnist_cnn_tda_seed0'))

gradcam_cnn = GradCAMOnCNN(cnn, layer_name='conv2d_3')
gradcam_cnn_tda = GradCAMOnCNNTDANet(cnn_tda, local_layer_name='conv2d_13', global_layer_name='conv1d_8')

## Creating GradCAM-guided images
1. Compute GradCAM
2. Get binarzied image.
3. Multiply image with the binarized image

In [ ]:
X_img_cnn = np.zeros_like(X)
X_img_cnn_tda = np.zeros_like(X)

start = time.time()
for i, (img, tda, label) in enumerate(zip(X, X_tda, y)):
    print(f"[{i+1:>5}/{len(X):>5}] Elapsed time: {time.time() - start:.2f}", end='\r')
    gradcam_cnn.to_heatmap(img)
    heatmap_cnn = gradcam_cnn._cache['heatmap'][0] / 255.0
    binarized_heatmap_cnn = (heatmap_cnn > 0.2).astype(float)
    X_img_cnn[i] = img * binarized_heatmap_cnn

    gradcam_cnn_tda.to_heatmap([img[np.newaxis], tda[np.newaxis]])
    heatmap_cnn_tda = gradcam_cnn_tda._cache['heatmap'][0] / 255.0
    binarized_heatmap_cnn_tda = (heatmap_cnn_tda > 0.2).astype(float)
    X_img_cnn_tda[i] = img * binarized_heatmap_cnn_tda


In [ ]:
np.save(os.path.join(dir_target, 'cnn3.npy'), X_img_cnn)
np.save(os.path.join(dir_target, 'cnn_tda3.npy'), X_img_cnn_tda)